In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from metrics import dice_loss, dice_coef, iou

In [2]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [3]:
""" Global parameters """
H = 512
W = 512

In [4]:
def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "CXR_png", "*.png")))
    masks1 = sorted(glob(os.path.join(path, "ManualMask", "leftMask", "*.png")))
 

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y1, valid_y1 = train_test_split(masks1, test_size=split_size, random_state=42)
 

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y1, test_y1 = train_test_split(train_y1, test_size=split_size, random_state=42)
 

    return (train_x, train_y1 ), (valid_x, valid_y1 ), (test_x, test_y1 )

In [6]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [25]:
def read_mask(path1):
    x1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    
    x = x1
    x = cv2.resize(x, (W, H))
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [26]:
def tf_parse(x, y1 ):
    def _parse(x, y1 ):
        x = x.decode()
        y1 = y1.decode()
         

        x = read_image(x)
        y = read_mask(y1)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y1], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [27]:
def tf_dataset(X, Y1, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y1))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset

In [28]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

In [29]:
""" Directory for storing files """
create_dir("LeftMaskUPTfiles")

In [30]:
""" Hyperparameters """
batch_size = 2
lr = 1e-5
num_epochs = 30
model_path = os.path.join("LeftMaskUPTfiles", "model.h5")
csv_path = os.path.join("LeftMaskUPTfiles", "data.csv")

In [31]:
""" Dataset """
dataset_path = "MontgomerySet"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)} ")
print(f"Valid: {len(valid_x)} - {len(valid_y)} ")
print(f"Test: {len(test_x)} - {len(test_y)} ")


Train: 112 - 112 
Valid: 13 - 13 
Test: 13 - 13 


In [32]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

## Bulid Model

In [33]:
len(train_dataset), len(valid_dataset)

(56, 7)

In [34]:

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [35]:
input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_57 (Conv2D)             (None, 512, 512, 64  1792        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 512, 512, 64  256        ['conv2d_57[0][0]']              
 ormalization)                  )                                                             

 max_pooling2d_15 (MaxPooling2D  (None, 32, 32, 512)  0          ['activation_61[0][0]']          
 )                                                                                                
                                                                                                  
 conv2d_65 (Conv2D)             (None, 32, 32, 1024  4719616     ['max_pooling2d_15[0][0]']       
                                )                                                                 
                                                                                                  
 batch_normalization_62 (BatchN  (None, 32, 32, 1024  4096       ['conv2d_65[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_62 (Activation)     (None, 32, 32, 1024  0           ['batch_normalization_62[0][0]'] 
          

 batch_normalization_69 (BatchN  (None, 256, 256, 12  512        ['conv2d_72[0][0]']              
 ormalization)                  8)                                                                
                                                                                                  
 activation_69 (Activation)     (None, 256, 256, 12  0           ['batch_normalization_69[0][0]'] 
                                8)                                                                
                                                                                                  
 conv2d_transpose_15 (Conv2DTra  (None, 512, 512, 64  32832      ['activation_69[0][0]']          
 nspose)                        )                                                                 
                                                                                                  
 concatenate_15 (Concatenate)   (None, 512, 512, 12  0           ['conv2d_transpose_15[0][0]',    
          

In [19]:
from tensorflow.keras.utils import CustomObjectScope

In [36]:
#with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
model = build_unet((H, W, 3))
metrics = [dice_coef, iou, Recall(), Precision()]

In [21]:
mse_loss = tf.keras.losses.MeanSquaredError()

In [37]:
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)


In [38]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path)
]


In [39]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

Epoch 1/30


2022-09-22 21:53:01.943788: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


56/56 [==============================] - ETA: 0s - loss: 0.6974 - dice_coef: 0.3026 - iou: 0.1792 - recall_1: 0.8538 - precision_1: 0.2512
Epoch 1: val_loss improved from inf to 0.78204, saving model to LeftMaskUPTfiles/model.h5
56/56 [==============================] - 42s 570ms/step - loss: 0.6974 - dice_coef: 0.3026 - iou: 0.1792 - recall_1: 0.8538 - precision_1: 0.2512 - val_loss: 0.7820 - val_dice_coef: 0.2134 - val_iou: 0.1201 - val_recall_1: 0.9933 - val_precision_1: 0.1452 - lr: 1.0000e-05
Epoch 2/30
56/56 [==============================] - ETA: 0s - loss: 0.5854 - dice_coef: 0.4146 - iou: 0.2637 - recall_1: 0.9347 - precision_1: 0.3337
Epoch 2: val_loss improved from 0.78204 to 0.77133, saving model to LeftMaskUPTfiles/model.h5
56/56 [==============================] - 33s 592ms/step - loss: 0.5854 - dice_coef: 0.4146 - iou: 0.2637 - recall_1: 0.9347 - precision_1: 0.3337 - val_loss: 0.7713 - val_dice_coef: 0.2252 - val_iou: 0.1278 - val_recall_1: 0.9952 - val_precision_1: 0.146

56/56 [==============================] - 32s 573ms/step - loss: 0.1990 - dice_coef: 0.8010 - iou: 0.6698 - recall_1: 0.9869 - precision_1: 0.9708 - val_loss: 0.2171 - val_dice_coef: 0.7847 - val_iou: 0.6496 - val_recall_1: 0.9795 - val_precision_1: 0.9442 - lr: 1.0000e-05
Epoch 18/30
56/56 [==============================] - ETA: 0s - loss: 0.1974 - dice_coef: 0.8026 - iou: 0.6724 - recall_1: 0.9881 - precision_1: 0.9726
Epoch 18: val_loss improved from 0.21413 to 0.20957, saving model to LeftMaskUPTfiles/model.h5
56/56 [==============================] - 33s 585ms/step - loss: 0.1974 - dice_coef: 0.8026 - iou: 0.6724 - recall_1: 0.9881 - precision_1: 0.9726 - val_loss: 0.2096 - val_dice_coef: 0.7916 - val_iou: 0.6572 - val_recall_1: 0.9708 - val_precision_1: 0.9676 - lr: 1.0000e-05
Epoch 19/30
56/56 [==============================] - ETA: 0s - loss: 0.1932 - dice_coef: 0.8068 - iou: 0.6777 - recall_1: 0.9881 - precision_1: 0.9712
Epoch 19: val_loss improved from 0.20957 to 0.20604, savi

In [40]:
""" Predicting the mask """
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracing the image name. """
    image_name = x.split("/")[-1]

    """ Reading the image """
    ori_x = cv2.imread(x, cv2.IMREAD_COLOR)
    ori_x = cv2.resize(ori_x, (W, H))
    x = ori_x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)

    """ Reading the mask """
    ori_y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    ori_y = cv2.resize(ori_y, (W, H))
    ori_y = np.expand_dims(ori_y, axis=-1)  ## (512, 512, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (512, 512, 3)

    """ Predicting the mask. """
    y_pred = model.predict(x)[0] > 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the predicted mask along with the image and GT """
    save_image_path = f"LeftMaskUPTfiles/Result/{image_name}"
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)

    sep_line = np.ones((H, 10, 3)) * 255

    cat_image = np.concatenate([ori_x, sep_line, ori_y, sep_line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_image)

  0%|                                                    | 0/13 [00:00<?, ?it/s]

1/1 [==============================] - 0s 294ms/step


  8%|███▍                                        | 1/13 [00:00<00:07,  1.69it/s]

1/1 [==============================] - 0s 20ms/step


 15%|██████▊                                     | 2/13 [00:00<00:04,  2.27it/s]

...

 85%|████████████████████████████████████▍      | 11/13 [00:04<00:00,  2.88it/s]

1/1 [==============================] - 0s 20ms/step


 92%|███████████████████████████████████████▋   | 12/13 [00:04<00:00,  2.94it/s]

1/1 [==============================] - 0s 20ms/step


100%|███████████████████████████████████████████| 13/13 [00:04<00:00,  2.76it/s]
